In [3]:
import pandas as pd
data = pd.read_csv("../../data_man/data_pre.csv")
data = data.drop(["Unnamed: 0"], axis=1)
data.head()

,Survived,Pclass,Female,Male,Age,SibSp,Parch,Fare,S,Q,C
0,0.0,1.0,0.0,1.0,0.271174,0.125,0.0,0.014151,1.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.472229,0.125,0.0,0.139136,0.0,0.0,1.0
2,1.0,1.0,1.0,0.0,0.321438,0.000,0.0,0.015469,1.0,0.0,0.0
3,1.0,0.0,1.0,0.0,0.434531,0.125,0.0,0.103644,1.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.434531,0.000,0.0,0.015713,1.0,0.0,0.0


In [4]:
from sklearn.model_selection import train_test_split
X, Y = data.drop(["Survived"], axis=1), data["Survived"]
x, x_test, y, y_test = train_test_split(X,Y,test_size=0.2,train_size=0.8)
x_train, x_cv, y_train, y_cv = train_test_split(x,y,test_size = 0.25,train_size =0.75)

# Linear Classifier Model

In [5]:
from sklearn.linear_model import RidgeClassifier
regr = RidgeClassifier()

In [6]:
regr.fit(x_train,y_train)
regr.score(x_cv,y_cv)

0.8033707865168539

In [7]:
regr.fit(x,y)
regr.score(x_test,y_test)

0.8100558659217877

# MLP Classifier

In [9]:
from sklearn.neural_network import MLPClassifier
perf = []
for i in range(1,100):
    mlp_class = MLPClassifier(hidden_layer_sizes=(10,i*2,i*4), max_iter=1000)
    mlp_class.fit(x_train, y_train)
    perf.append((mlp_class.score(x_cv,y_cv),i))
max(perf)

In [10]:
mlp_class = MLPClassifier(hidden_layer_sizes=(10,62*2,62*4), max_iter=1000)
mlp_class.fit(X, Y)
mlp_class.score(x_test, y_test)

0.8435754189944135

# Using Feature Selection to get even better results 

In [21]:
final = []
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
X_new = SelectKBest(f_classif, k=1).fit_transform(x,y)
mlp_class = MLPClassifier(hidden_layer_sizes=(10,62*2,62*4), max_iter=1000)
mlp_class.fit(X_new, y)
X_new_test = SelectKBest(f_classif, k=1).fit_transform(x_test,y_test)
final.append((mlp_class.score(X_new_test, y_test),i))

In [25]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
final = []
for i in range(1, 11):
    X_new = SelectKBest(f_classif).fit_transform(x,y)
    mlp_class = MLPClassifier(hidden_layer_sizes=(10,62*2,62*4), max_iter=1000)
    mlp_class.fit(X_new, y)
    X_new_test = SelectKBest(f_classif).fit_transform(x_test,y_test)
    final.append((mlp_class.score(X_new_test, y_test),i))
print(final)

[(0.8212290502793296, 1), (0.8044692737430168, 2), (0.8156424581005587, 3), (0.8156424581005587, 4), (0.8044692737430168, 5), (0.8212290502793296, 6), (0.7988826815642458, 7), (0.8212290502793296, 8), (0.8044692737430168, 9), (0.8268156424581006, 10)]


In [27]:
mlp_class = MLPClassifier(hidden_layer_sizes=(10,62*2,62*4), max_iter=1000)
from sklearn.feature_selection import SequentialFeatureSelector
sfs = SequentialFeatureSelector(mlp_class)
sfs.fit(x_train,y_train)

SequentialFeatureSelector(estimator=MLPClassifier(hidden_layer_sizes=(10, 124,
                                                                      248),
                                                  max_iter=1000))

In [28]:
sfs.get_feature_names_out()

array(['Pclass', 'Female', 'Age', 'Parch', 'S'], dtype=object)

In [29]:
X_new = sfs.transform(X)

In [33]:
mlp_class.fit(X_new, Y)

MLPClassifier(hidden_layer_sizes=(10, 124, 248), max_iter=1000)

# Using PCA (it sucks)

In [37]:
from sklearn.decomposition import PCA

In [38]:
final = []
for i in range(1, 11):
    pca = PCA(i)
    pca.fit(x_train,y_train)
    final.append(pca.score(x_test,y_train))
print(final)

[-2.0684382310437623, -0.8310688693098204, 0.9336424581919451, 2.7399623254542576, 3.1401056204864104, 3.419651541376044, 3.754899866021287, -inf, -47363784302912.98, -43135696450512.016]


# Using Better Model for Predictions on test dataset

In [34]:
pred_data = pd.read_csv("../../data_man/pred_data.csv")
pred_data = pred_data.drop(["Unnamed: 0"], axis=1)
pred_data.head()

,Pclass,Female,Male,Age,SibSp,Parch,Fare,S,Q,C
0,1.0,0.0,1.0,0.452723,0.000,0.000000,0.015282,0.0,1.0,0.0
1,1.0,1.0,0.0,0.617566,0.125,0.000000,0.013663,1.0,0.0,0.0
2,0.5,0.0,1.0,0.815377,0.000,0.000000,0.018909,0.0,1.0,0.0
3,1.0,0.0,1.0,0.353818,0.000,0.000000,0.016908,1.0,0.0,0.0
4,1.0,1.0,0.0,0.287881,0.125,0.111111,0.023984,1.0,0.0,0.0


In [35]:
pred_data = sfs.transform(pred_data)
pred = mlp_class.predict(pred_data)

In [36]:
print(list(pred))
final = []
pred_csv = pd.read_csv("../../data_man/predictions.csv")
pred_csv = pred_csv.drop(["Unnamed: 0"], axis=1)
for i in range(len(list(pred))):
    final.append(int(pred[i]))
pred_csv.insert(1, "Survived", final, True)
pred_csv.to_csv("pred.csv",index=False)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0,